<a href="https://colab.research.google.com/github/vlasova-us/Python_public/blob/main/%D0%94%D0%BE%D0%BC%D0%B0%D1%88%D0%BD%D0%B5%D0%B5_%D0%B7%D0%B0%D0%B4%D0%B0%D0%BD%D0%B8%D0%B5_%D0%BA_%D0%BB%D0%B5%D0%BA%D1%86%D0%B8%D0%B8_%C2%AB%D0%9E%D1%81%D0%BD%D0%BE%D0%B2%D1%8B_%D0%B2%D0%B5%D0%B1_%D1%81%D0%BA%D1%80%D0%B0%D0%BF%D0%B8%D0%BD%D0%B3%D0%B0%C2%BB.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Вам необходимо написать функцию, которая будет основана на поиске по сайту habr.com. Функция в качестве параметра должна принимать список запросов для поиска (например, ['python', 'анализ данных']) и на основе материалов, попавших в результаты поиска по каждому запросу, возвращать датафрейм вида:

<дата> - <заголовок> - <ссылка на материал>

В рамках задания предполагается работа только с одной (первой) страницей результатов поисковой выдачи для каждого запроса. Материалы в датафрейме не должны дублироваться, если они попадали в результаты поиска для нескольких запросов из списка.



In [ ]:
import time
import pandas as pd
import requests
from bs4 import BeautifulSoup
import re
import numpy as np

In [ ]:
def get_all_links(query, pages):
  ''' 
Функция принимает на вход список из поискового запроса(ов) (query) для поиска на сайте https://habr.com , а также глубину поиска (количество поисковых страниц), которую необходимо вывести (pages).
Функция возвращает список из ссылок.

  '''
  url = 'https://habr.com/ru/search/'
  all_links = []
  links_list = []
  params = {'q': query}

  for i in range(1, pages+1):
    if i == 1:
      req = requests.get(url, params)
    else: req = requests.get((str(url) + str('page')+str(i)+'/'), params)
   
    soup = BeautifulSoup (req.text)  
    time.sleep(0.33)
    refs = soup.find_all ('a', class_='tm-article-snippet__title-link')
    for ref in refs:
      all_links.append(ref.get('href'))
      all_full_links = list(map(lambda x: 'https://habr.com' + x, all_links))   #Ссылки
      links_list += all_full_links 
  return set(links_list)
    

In [ ]:
def search_habr (all_links): 
  '''
Функция принимает на вход список из ссылок, полученных функцией get_all_links, от поискового запроса(ов) (query) для поиска на сайте https://habr.com 
Функция возвращает DataFrame из столбцов:
- Дата опубликования статьи в datatime формате (колонка date)
- Заголовок статьи (колонка title)
- Ссылка на статью (колонка link)
- Полный текст статьи (колонка text)
- Количество лайков: общее кол-во, позитивные, негативные (колонка like)
  '''
  habr_articles_df=pd.DataFrame()
  for link in all_links: 
    soup = BeautifulSoup(requests.get(link).text)
    time.sleep(0.33)
    date = pd.to_datetime (soup.find ('span', class_='tm-article-snippet__datetime-published').find('time').get('datetime') ) #Дата в формате Datetime
    
    if soup.find ('div', class_='tm-article-snippet tm-article-presenter__snippet tm-article-snippet'): #Заголовок. 
     title = soup.find ('div', class_='tm-article-snippet tm-article-presenter__snippet tm-article-snippet').find('h1').find('span').text 
    elif soup.find ('div', class_='tm-articles-list'):
      title = soup.find ('div', class_='tm-articles-list').find('article'). find('div').find('h2').find('a').find('span').text
    else: title = np.nan
    
    if soup.find ('div', class_= 'tm-votes-meter tm-data-icons__item'): #Лайк
     like_all = soup.find ('div', class_= 'tm-votes-meter tm-data-icons__item').find('span').get('title')[14:] # По заданию не было условия разбить на три группы лайки: общее кол-во, позитивные и негативные, поэтому для сохранения данных оставила все 3 группы в одном столбце.
    elif soup.find ('div', class_='tm-votes-meter tm-article-rating__votes-switcher'):
     like_all = soup.find ('div', class_='tm-votes-meter tm-article-rating__votes-switcher').find('svg').text [14:]
    else:
     like_all = ''
    if like_all == '': #Если лайков у текста нет, то выставляется NaN. 
      like_all = np.nan

    if soup.find('div', class_='article-formatted-body article-formatted-body article-formatted-body_version-2'): #Полный текст
      text = soup.find('div', class_='article-formatted-body article-formatted-body article-formatted-body_version-2').text.strip().replace('\n', '')
    elif soup.find('div', class_='article-formatted-body article-formatted-body article-formatted-body_version-1'):
       text = soup.find ('div', class_='article-formatted-body article-formatted-body article-formatted-body_version-1').text.strip().replace('\n', '')
    else: text = np.nan
    
    row = {'date': date, 'title': title, 'link': link, 'text': text,'like': like_all}
    habr_articles_df = pd.concat ([habr_articles_df, pd.DataFrame([row]) ] ) #  Объединение в DataFrame всех строк
       
  return habr_articles_df.reset_index (drop = True)

all_links = get_all_links(['python', 'go'], 4)
habr_ = search_habr (all_links)
display(habr_)

,date,title,link,text,like
0,2021-04-04 09:12:01+00:00,Все важные фичи и изменения в Python 3.10,https://habr.com/ru/company/skillfactory/blog/...,Если вам хочется попробовать все фичи великоле...,28: ↑26 и ↓2
1,2022-05-24 07:57:47+00:00,Дайджест Слёрма: тест на уровень кунг-фу по Py...,https://habr.com/ru/company/southbridge/news/t...,Сделали для вас подборку свежих статей и выгод...,11: ↑10 и ↓1
2,2020-12-12 12:22:10+00:00,Написание WinLocker'а на Python,https://habr.com/ru/post/532684/,Внимание!Данная статья была создана в познават...,35: ↑11 и ↓24
3,2021-12-14 07:49:34+00:00,Go после Python: как я учу новый язык,https://habr.com/ru/company/avito/blog/594495/,"Привет, Хабр! Меня зовут Максим Чижов, я бэкен...",27: ↑23 и ↓4
4,2021-11-09 15:55:38+00:00,Диаграмма воронки в Python,https://habr.com/ru/company/otus/blog/588190/,ВведениеДиаграммы-воронки часто используются д...,3: ↑2 и ↓1
...,...,...,...,...,...
75,2014-10-03 08:03:48+00:00,Парсим на Python: Pyparsing для новичков,https://habr.com/ru/post/239081/,Парсинг (синтаксический анализ) представляет с...,57: ↑46 и ↓11
76,2020-03-11 12:48:45+00:00,Создание виртуальных окружений и установка биб...,https://habr.com/ru/post/491916/,Язык программирования Python считается достато...,13: ↑11 и ↓2
77,2021-10-25 09:51:23+00:00,Easy concurrency with Python Shared Object,https://habr.com/ru/post/585320/,Project repository.Year old article about gene...,1: ↑1 и ↓0
78,2019-11-13 11:25:06+00:00,Пишем чат на Python и Django,https://habr.com/ru/company/otus/blog/475672/,"Добрый день, друзья. В преддверии старта курса...",24: ↑16 и ↓8
